## Import libraries

In [1]:
## import libraries%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
sys.path.append('scripts/')
from ml_method import *
from proj1_helpers import *

from scripts.preprocess import *

%load_ext autoreload
%autoreload 2

## Load data

In [8]:
y, x, ind = load_csv_data('higgs-data/train.csv')

## Clean data and normalize data

In [9]:
mask = (x != -999)


# compute the mean and standard deviations
mean = (x * mask).sum(axis=0)/np.sum(mask, axis=0)
std_dev = np.sqrt((((x - mean) * mask)**2).sum(axis=0)/np.sum(mask, axis=0))

# normalize the data
stand_x = (x * mask - mean)/std_dev

# ------- standarization finish ------------


# set -999 to 0
stand_x[~mask] = 0

# --------- Setting -999 to 0 --------------


tmp = np.ones([stand_x.shape[0], stand_x.shape[1] + 1])
tmp[:,1:] = stand_x
stand_x = tmp

# ---------- add 1 to x  finish ---------------

## Define the method to use

In [10]:
def logistic(x, y, ratio):
    
    score = 0
    for i in range(5):
        index = np.array(range(x.shape[0]))
        np.random.shuffle(index)

        x = x[index]
        y = y[index]
        num = np.round(x.shape[0] * ratio).astype(np.int)

        # instantiate a logistic regression model, and fit with X and y
        model = LogisticRegression()
        model = model.fit(x[0:num], y[0:num])

        score += model.score(x[num:], y[num:])
    return score/5


## Feature selection

In [9]:
# for i in range(10,15):
#     x = getPCA(stand_x, i)
#     score = logistic(x,y, 0.8)
#     print('number: ', i, ', score: ', score)

highest_acc = 0
best_x = stand_x.copy()
worst_feat = 0
record =[]

for i in range(30):
    flag = 0
    for j in range(30 - i):
        x = best_x.copy()
        x = np.delete(x, j, 1)
        score = logistic(x,y, 0.7)
        if score > highest_acc:
            highest_acc = score
            worst_feat = j
            flag = 1
        print('number: ', j, ', score: ', score)
    if flag == 1:
        best_x = np.delete(best_x, worst_feat, 1)
        print('remove feature', worst_feat)
        record.append(worst_feat)
    else:
        break

print('best score: ', highest_acc)


number:  0 , score:  0.749853333333
number:  1 , score:  0.750258666667
number:  2 , score:  0.731717333333
number:  3 , score:  0.741512
number:  4 , score:  0.747224
number:  5 , score:  0.751002666667
number:  6 , score:  0.74992
number:  7 , score:  0.750650666667
number:  8 , score:  0.734261333333
number:  9 , score:  0.749136
number:  10 , score:  0.751312
number:  11 , score:  0.747616
number:  12 , score:  0.753264
number:  13 , score:  0.747738666667
number:  14 , score:  0.749949333333
number:  15 , score:  0.750784
number:  16 , score:  0.749781333333
number:  17 , score:  0.750722666667
number:  18 , score:  0.749850666667
number:  19 , score:  0.750933333333
number:  20 , score:  0.750466666667
number:  21 , score:  0.751690666667
number:  22 , score:  0.750077333333
number:  23 , score:  0.749629333333
number:  24 , score:  0.750434666667
number:  25 , score:  0.751098666667
number:  26 , score:  0.750213333333
number:  27 , score:  0.750832
number:  28 , score:  0.75014

In [11]:
best_x = stand_x.copy()
record = [12, 14]
for i in record:
    best_x = np.delete(best_x, i, 1)

# score = logistic(best_x,y,0.7)
# score

In [12]:
best_x.shape

(250000, 29)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

x_tr, x_te, y_tr, y_te = train_test_split(best_x, y, test_size = 0.8, random_state = 0)

# x_tr.shape
# y_tr.shape

clf = svm.SVC(kernel='rbf', C=1).fit(x_tr, y_tr)
print(clf.score(x_te, y_te))

for i in range(25,29):
    x1 = best_x.copy()
    x1 = getPCA(x1, i)
    x_tr, x_te, y_tr, y_te = train_test_split(x1, y, test_size = 0.8, random_state = 0)

    # x_tr.shape
    # y_tr.shape

    clf = svm.SVC(kernel='rbf', C=1).fit(x_tr, y_tr)
    print(clf.score(x_te, y_te))


0.828
0.828275
0.828125
0.8281
0.828055


In [12]:
clf.score(x_te, y_te)
83296

0.82918999999999998